<a href="https://colab.research.google.com/github/jbloewencolon/The-Demographics-of-Faerun/blob/main/DnD_Dataset_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import random

In [9]:
# Define the file path (it seems you've already specified it)
file_path = '/content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx'

In [10]:
column_names = [
    "City",
    "Region",
    "Nation",
    "Government",
    "Percentage of population Humans",
    "Percentage of population Halflings",
    "Percentage of population Orcs",
    "Percentage of population Elves",
    "Percentage of population Dwarves",
    "Percentage of population Gnomes",
    "Average Age of Population",
    "Typical Cause of Death",
    "Wizarding School",
    "Active Economy",
    "Hidden Economy",
    "Known Treasure Value",
    "Rumored Treasure Value",
    "Major Industries",
    "Tax Rate",
    "Major Resources",
    "# of Landowners",
    "# Homeowners",
    "Average Rent",
    "# of Peasants",
    "Average Debt by Household",
    "# of Nobility",
    "# of Active Adventurers",
    "Number of Active Wizards",
    "Dragon Sightings",
    "Environmental Climate",
    "Average Temperature",
    "Average Rainfall",
    "Political Climate",
    "Magical Climate",
    "Likelihood of Getting Cursed",
    "Danger Level",
    "Openness to Strangers",
    "Notable Landmarks",
    "Fate ranking",
    "Total Gold in Region"
]

df = pd.DataFrame(columns=column_names)

In [ ]:
print(df.isna().sum())  # Print count of NaN values in each column
print(df.isin([np.inf, -np.inf]).sum())  # Print count of infinite values in each column

In [47]:
def generate_dataset(seed=42):
    np.random.seed(seed)

    # Initialize constants
    num_cities = 411
    species = ["Humans", "Halflings", "Orcs", "Elves", "Dwarves", "Gnomes"]
    columns = ["City", "Region", "Nation", "Government Type"] + [f"Percentage of population {s}" for s in species] + [
        "Average Age of Population", "Typical Cause of Death", "Wizarding School", "Active Economy (Gold)",
        "Hidden Economy (Gold)", "Known Treasure Value", "Rumored Treasure Value", "Dragon Sightings", "Tax Rate",
        "Major Industries", "Major Resources", "Number of Landowners", "Number of Homeowners", "Average Rent",
        "Number of Peasants", "Average Debt by Household", "Number of Nobility", "Number of Active Adventurers",
        "Number of Active Wizards", "Likelihood of Getting Cursed", "Danger Level", "Openness to Strangers",
        "Notable Landmarks", "Environmental Climate", "Average Temperature", "Average Rainfall", "Political Climate",
        "Magical Climate", "Fate Ranking", "Total Gold in Region"
    ]
    city_names = [
    "Abriymoch", "Aelor", "Ageadren", "Airspur", "Airspur (Chessenta)", "Akanax",
    "Akkaido", "Al-Anwahr", "Alashan", "Almraiven", "Ammathluir", "Ammathtar",
    "Anaxadren", "Araugul", "Ariel-than", "Asple", "Assam", "Athkatla",
    "Aventine", "Azumar", "Balanzia", "Baldur's Gate", "Beluir", "Beradandar",
    "Berydren", "Bhutan", "Bidnop So", "Blingdenstone", "Bral", "Branestria",
    "Callidae", "Calimport", "Castle Darkhold", "Cathyr", "Cedarsproke", "Chaoyang",
    "Chaul'mur'ssin", "Chavyondat", "Ched Nasad", "Cheju", "Choshein", "Chrysalis",
    "Chunming", "Chunming (T'u Lung)", "Cimbar", "Citadel of Ice and Steel",
    "Citadel of Ten Thousand Pearls", "City of Brass", "City of Caverns", "City of Glass",
    "City of the Werecrocodiles", "Clorlyth'na'tsheen", "Corrant", "Dalekeva",
    "Dalelost", "Darka", "Darmshall", "Darrawn", "Dashadjen", "Dauntshield",
    "Dawnfire", "Deepearth", "Delia", "Delselar", "Delzimmer", "Derlusk",
    "Dhinnilith", "Dilpur", "Djerad Kethendi", "Djerad Kusold", "Dloemen", "Dojyu",
    "Dolblunde", "Dralpur", "Drelagara", "Duhlnarim", "Dunspeirrin", "Dusklan",
    "Dyon G'ennivalz", "Elgrin Fau", "Ereshkigal", "Evereska", "Everlund", "Evernight",
    "Eylea", "Faenasuor", "Fatula Chupa", "Fij", "Fluvenilstra", "Flyndagol",
    "Fochu", "Foo Nakar", "Forharn", "Fukiow", "Gharreil", "Gheldaneth",
    "Gifu", "Gizekhtet (city)", "Glister", "Gloomwrought", "Gnarhelm (city)", "Goliad",
    "Gontal City", "Guallidurth", "Gundbarg", "Gurdeth", "Halagard", "Hammerstaad",
    "Harglast", "Haunvaerune", "Hazuth", "Helbrester", "Heldapan", "Helgabal",
    "Helm's Hold", "Herath", "Hethar", "Hilbrand", "Hisari", "Hlammach",
    "Huorm", "Iiso", "Ilimar", "Iliphanar", "Illul", "Iltkazar",
    "Imdolphyn", "Immurk's Hold", "In'ani", "Inupras", "Iriaebor", "Ironfang Deep",
    "Ironspur", "Ithmong (Lapaliiya)", "Ixtzul", "Jangling Hiter", "Jasuga", "Jhachalkhyn",
    "Jhalhoran", "Jhothûn (city)", "Joya", "Jugicha", "Kanaglym", "Karak",
    "Karffbadh", "Kaspar", "Keltar", "Khaerbaal", "Khampo", "Koje",
    "Konang Pen", "Konigheim", "Kormul", "Kourmira", "Kressilacc", "Kuda",
    "Kuo Te' Lung", "Kushk", "Lasdur", "Lheshayl", "Lipo",
    "Lith My'athar", "Llorbauth", "Llurth Dreier", "Lothen of the Silver Spires", "Luenath", "Lundeth (Thindol)",
    "Lushpool", "Luskan", "Lylorn", "Maerlar", "Malduir", "Malsheem",
    "Malydren", "Manass", "Marabaya", "Marangor", "Marrauk", "Maxal",
    "Melvaunt", "Messemprar", "Mhairhetel", "Mictlatepec", "Mithral Hall",
    "Minauros (city)", "Mirabar", "Mordulkin", "Morvar", "Motra", "Mreyelundar",
    "Mulmaster", "Multum", "Mussum", "Myrmyr", "Myth Adofhaer", "Myth Drannor",
    "Myth Nantar", "Nakamaru", "Narubel", "Nathong", "Neldorild", "Nesmé",
    "Nethjet", "Nethra", "Neverwinter", "New Sarshel", "Nivian", "Nonthal",
    "Northkeep", "Nycopolis", "Obbolor", "Oberon's Hold", "Occidian", "Oeble",
    "Ojy-do", "Okahira", "Old Stone Keep", "Old Vaelen", "Omu", "Ooltul",
    "Opawangchicheng", "Orlumbor", "Ormath", "Ormpetarr", "Ornthalas", "Oryndoll",
    "Osel", "Ossantil", "Oth", "Outentown", "Ozryn", "Paolan",
    "Paraelemental City of Conflagration", "Paraelemental City of Deluge", "Paraelemental City of Ooze",
    "Paraelemental City of Sleet", "Phandalin", "Phannaskul",
    "Phaervorul", "Phrenopolis", "Piaoliang", "Pinebluff", "Pirth City", "Piyapon",
    "Plague-Mort", "Plotsklan", "Port Castigliar", "Port Llast", "Port Nyanzaru", "Port of Ravens Bluff",
    "Poytakht", "Priapurl", "Procampur", "Purskul", "Pyarados", "Q'barra",
    "Q'Xorlarrin", "Qazat", "Quaervarr", "Quarmall", "Qu'ellar d'Uzunt", "Qu'ellarz'orl",
    "Radocan", "Raurokh", "Raven's Bluff", "Razalpartha", "Rekkar-Sarrat", "Relkath's Foot",
    "Reth", "Revonar", "Riatavin", "Rilauven", "Ruathym (city)", "Ruldoer",
    "Rylatar", "Saelmur", "Sakkors", "Saradush", "Sardolphyn", "Sarth",
    "Sattar", "Scardale", "Schamedar", "Schorin", "Secomber", "Sekras",
    "Selgaunt", "Semkhrun", "Sepech", "Serôs (city)", "Setma'antar", "Sethlapurkha",
    "Shaboath", "Shade (city)", "Shadusk", "Shangban", "Shaundalar", "Sheirtalar",
    "Shi-Kuo", "Shindikuur", "Shoonach", "Shu Pao", "Shussel", "Sildeyuir",
    "Siluvanede", "Simbia", "Sintharla", "Sjorndorn", "Skullport", "Skuld",
    "Soorenar", "Sschindylryn", "Sshamath", "Stornanter", "Sudar", "Sundabar",
    "Sunduluth", "Sungping", "Surkh", "Suzail", "Svartalfheim", "Taejin",
    "Tagorlad", "Takalai", "Takar", "Talab", "Talath Passar", "Talath Seren",
    "Talbot", "Talhund", "Talinie (city)", "Tammar", "Tantras", "Tasseldale",
    "Tchazzar", "Telantiwar", "Telflamm", "Telnarquel", "Telos City", "Telsorn",
    "Temar", "Termalaine", "Tethgard", "Thallar", "Thamor", "Thangorodrim",
    "Tharqualnaar", "Thayd", "Thelnam", "Thentia", "Thesk", "Thulanth",
    "Thultanthar", "Thundarlun", "Thundertree", "Thyat", "Tibet", "Tilverton",
    "Tith Tilendrothael", "Torsil", "Triboar", "Triel", "Tsurlagol", "Tu Pe",
    "Tulmon", "Tun Mi Lung", "Tuncer", "Tunfeng", "Tunzel", "Turmish (city)",
    "Twilight Hall", "Two Stars", "Tyar-Besil", "Ulgarth (city)", "Ulgarstat", "Umbergoth",
    "Umbraer", "Urkhar", "Urossos", "Uthmere", "Uzurr", "Vaasa (city)",
    "Valjevo Castle", "Vangal", "Velegrar", "Vilhon Reach", "Volothamp", "Voonlar",
    "Vor Rukoth", "V'ral", "Waterdeep", "Weeping War (city)", "Westgate", "Wheloon",
    "Xarcorr", "Xinlenal", "Xonthal's Tower", "Xynnar", "Yal Tengri", "Yaulazna",
    "Yhaunn", "Yhro 'Lamith", "Ylraphon", "Yuirwood", "Yunn Shicheng", "Za Si Tsan",
    "Zazesspur", "Zhentil Keep", "Zhenul", "Zindalankh", "Zokir"
    ]
    region_names = ["Region1", "Region2"]  # Add a list of region names here
    nation_names = ["Nation1", "Nation2"]  # Add a list of nation names here
    government_types = ["Monarchy", "Democracy", "Oligarchy", "Theocracy"]
    causes_of_death = ["Old Age", "Disease", "Conflict", "Accident"]
    major_industries = ["Mining", "Trade", "Farming", "Craftsmanship"]
    major_resources = ["Gold", "Iron", "Wood", "Stone"]
    notable_landmarks = ["Landmark1", "Landmark2"]  # Add a list of notable landmarks here
    species_age_ranges = {
            "Humans": [1, 100],
            "Dwarves": [1, 350],
            "Elves": [1, 750],
            "Gnomes": [300, 500],
            "Halflings": [1, 150],
            "Orcs": [1, 50]
        }
    df = pd.DataFrame(columns=columns)

    # Variables and functions definition section
    total_population = np.random.randint(66e6, 70e6)
    total_economy = np.random.uniform(80e6, 100e6)

    def select_industries_and_resources(industries, resources):
        selected_industries = np.random.choice(industries, size=np.random.randint(1, len(industries)), replace=False)
        selected_resources = np.random.choice(resources, size=np.random.randint(1, len(resources)), replace=False)
        return selected_industries, selected_resources

    def generate_population_distribution(species_age_ranges):
        distribution = {}
        for species, age_range in species_age_ranges.items():
            distribution[species] = np.random.randint(age_range[0], age_range[1])
        return distribution

    def calculate_average_age(row):
        weighted_ages = sum([(row[f'Percentage of population {species}'] / 100) * np.random.uniform(*species_age_ranges[species]) for species in species_age_ranges])
        return weighted_ages

    def calculate_dragon_sightings(row):
        region_gold = row['Total Gold in Region']
        max_sightings = min(50, row['Fate Ranking'] + region_gold // 1e6)
        return np.random.randint(0, max_sightings + 1)

    def adjust_danger_level(row):
        danger_level = row['Danger Level']
        if row['Percentage of population Orcs'] > 0:
            danger_level += 1
        if row['Percentage of population Elves'] > 0:
            danger_level -= 1
        return min(max(1, danger_level), 10)

    def calculate_magical_climate(row):
        return row['Dragon Sightings'] * 0.1 + row['Number of Active Wizards'] * 0.5

    def check_data_consistency(data):
        # Adjusting Population Distribution to 100
        total_distribution = sum(data["Population Distribution"].values())
        if total_distribution != 100:
            for key in data["Population Distribution"]:
                data["Population Distribution"][key] = (data["Population Distribution"][key] / total_distribution) * 100

        # Adjusting Number of Homeowners
        if data["Number of Homeowners"] > data["Number of Landowners"]:
            data["Number of Homeowners"] = data["Number of Landowners"]

    def calculate_openness_to_strangers(row):
        openness = row['Openness to Strangers']
        if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0:
            openness -= 2
        return min(max(0, openness), 10)

    for i in range(num_cities):
        city_population = np.random.randint(1000, 10000)
        species_percentages = np.random.dirichlet(np.ones(len(species)),size=1)[0] * 100

        data = {
            "City": np.random.choice(city_names),
            "Region": np.random.choice(region_names),
            "Nation": np.random.choice(nation_names),
            "Government Type": np.random.choice(government_types),
            "Population of City": city_population,
            "Population Distribution": generate_population_distribution(species_age_ranges),
            "Average Age of Population": np.random.choice([np.random.uniform(*species_age_ranges[spec]) for spec in species]),
            "Typical Cause of Death": np.random.choice(causes_of_death),
            "Wizarding School": np.random.choice(["Yes", "No"]),
            "Active Economy (Gold)": np.random.uniform(1e6, 10e6),
            "Hidden Economy (Gold)": np.random.uniform(1e6, 10e6),
            "Known Treasure Value": np.random.uniform(1e6, 10e6),
            "Rumored Treasure Value": np.random.uniform(1e6, 10e6),
            "Dragon Sightings": np.random.randint(0, 50),
            "Tax Rate": np.random.uniform(0.1, 0.8),
            "Major Industries": np.random.choice(major_industries),
            "Major Resources": np.random.choice(major_resources),
            "Number of Landowners": np.random.randint(1, 200),
            "Number of Homeowners": np.random.randint(10, 10000),
            "Average Rent": np.random.randint(1, 5000),
            "Number of Peasants": np.random.randint(10, 50000),
            "Average Debt by Household": np.random.uniform(0, 100000),
            "Number of Nobility": np.random.randint(0, 500),
            "Number of Active Adventurers": np.random.randint(0, 200),
            "Number of Active Wizards": np.random.randint(0, 20),
            "Likelihood of Getting Cursed": np.random.uniform(0, 10),
            "Danger Level": np.random.uniform(0, 10),
            "Openness to Strangers": np.random.uniform(0, 100),
            "Notable Landmarks": np.random.choice(notable_landmarks),
            "Environmental Climate": np.random.choice(["Tropical", "Temperate", "Arctic", "Desert", "Tundra"]),
            "Average Temperature": np.random.uniform(-30, 50),
            "Average Rainfall": np.random.uniform(0, 3000),
            "Political Climate": np.random.uniform(0, 100),
            "Magical Climate": np.random.uniform(0, 10),
            "Fate Ranking": np.random.uniform(0, 10),
            "Total Gold in Region": total_economy
        }
        # Check and adjust data consistency
        check_data_consistency(data)

        # Setting the species percentage of population
        for idx, spec in enumerate(species):
            data[f"Percentage of population {spec}"] = species_percentages[idx]

        # Adjust danger level and magical climate after setting the species percentage of population
        data["Danger Level"] = adjust_danger_level(data)
        data["Magical Climate"] = calculate_magical_climate(data)

        # Setting the species percentage of population
        for idx, spec in enumerate(species):
            data[f"Percentage of population {spec}"] = species_percentages[idx]

        # Adding the generated data to the DataFrame
        df = df.append(data, ignore_index=True)

    # Grouping operations and more data manipulation
    grouped = df.groupby('Region')
    region_gold_mapping = grouped[['Active Economy (Gold)', 'Hidden Economy (Gold)']].sum().sum(axis=1)
    df['Total Gold in Region'] = df['Region'].map(region_gold_mapping)

    # Applying the functions
    df['Dragon Sightings'] = df.apply(calculate_dragon_sightings, axis=1)
    df['Dragon Sightings'] = ((df['Dragon Sightings'] - df['Dragon Sightings'].min()) / (df['Dragon Sightings'].max() - df['Dragon Sightings'].min()) * 50).astype(int)
    df['Average Age of Population'] = df.apply(calculate_average_age, axis=1)
    df['Average Age of Population'] -= df['Danger Level'] * 0.5
    df['Average Age of Population'] = df['Average Age of Population'].clip(1)
    df['Hidden Economy'] = df['Number of Nobility'] * 1e4
    df['Openness to Strangers'] = df['Openness to Strangers'].clip(0, 10)

    # Adjusting population distribution for specific species across cities
    def adjust_population_distribution(df, species, percentage, city_count):
        total_population = df['Population of City'].sum()
        target_population = int(total_population * (percentage / 100))
        selected_cities = df.sample(city_count)
        each_city_population = target_population // city_count
        for i, city in selected_cities.iterrows():
            df.at[i, f'Percentage of population {species}'] = (each_city_population / city['Population of City']) * 100

    return df

# Call the function to generate the dataset and assign it to df
df = generate_dataset()

<ipython-input-47-0e4243e973c3>:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-47-0e4243e973c3>:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-47-0e4243e973c3>:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-47-0e4243e973c3>:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-47-0e4243e973c3>:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 43 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   City                                411 non-null    object 
 1   Region                              411 non-null    object 
 2   Nation                              411 non-null    object 
 3   Government Type                     411 non-null    object 
 4   Percentage of population Humans     411 non-null    float64
 5   Percentage of population Halflings  411 non-null    float64
 6   Percentage of population Orcs       411 non-null    float64
 7   Percentage of population Elves      411 non-null    float64
 8   Percentage of population Dwarves    411 non-null    float64
 9   Percentage of population Gnomes     411 non-null    float64
 10  Average Age of Population           411 non-null    float64
 11  Typical Cause of Death              411 non-n

In [49]:
# Save the DataFrame to Excel
df.to_excel(file_path, index=False)

# Confirm that the file has been saved
print(f"DataFrame saved to {file_path}")

DataFrame saved to /content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx


In [50]:
# Read the xlsx file
df = pd.read_excel(file_path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 43 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   City                                411 non-null    object 
 1   Region                              411 non-null    object 
 2   Nation                              411 non-null    object 
 3   Government Type                     411 non-null    object 
 4   Percentage of population Humans     411 non-null    float64
 5   Percentage of population Halflings  411 non-null    float64
 6   Percentage of population Orcs       411 non-null    float64
 7   Percentage of population Elves      411 non-null    float64
 8   Percentage of population Dwarves    411 non-null    float64
 9   Percentage of population Gnomes     411 non-null    float64
 10  Average Age of Population           411 non-null    float64
 11  Typical Cause of Death              411 non-n

In [51]:
df.head()

,City,Region,Nation,Government Type,Percentage of population Humans,Percentage of population Halflings,Percentage of population Orcs,Percentage of population Elves,Percentage of population Dwarves,Percentage of population Gnomes,...,Environmental Climate,Average Temperature,Average Rainfall,Political Climate,Magical Climate,Fate Ranking,Total Gold in Region,Population of City,Population Distribution,Hidden Economy
0,Gloomwrought,Region2,Nation2,Oligarchy,28.378268,19.675575,3.655725,3.655109,1.289634,43.345689,...,Tundra,-11.528494,723.076398,68.326352,5.8,8.331949,2.167492e+09,6390,"{'Humans': 1.877133105802048, 'Dwarves': 26.36...",1660000
1,Skuld,Region1,Nation1,Democracy,42.223666,5.267491,19.107342,6.570720,12.913080,13.917701,...,Temperate,-20.730475,2589.310278,62.329813,6.3,0.635584,2.451635e+09,9226,"{'Humans': 10.843373493975903, 'Dwarves': 5.30...",1380000
2,Iriaebor,Region1,Nation1,Democracy,23.394100,23.208338,18.940563,6.799198,17.438517,10.219284,...,Tropical,-8.229420,1943.070362,0.052038,7.1,3.047813,2.451635e+09,3454,"{'Humans': 1.232394366197183, 'Dwarves': 24.91...",3390000
3,Ulgarth (city),Region2,Nation2,Theocracy,14.562482,11.005711,5.116438,2.600931,8.390840,58.323598,...,Tropical,-17.942596,1524.596330,69.581281,6.3,3.259589,2.167492e+09,9901,"{'Humans': 4.293971924029727, 'Dwarves': 9.331...",2540000
4,Ched Nasad,Region2,Nation2,Monarchy,18.373859,24.532712,6.343358,1.496149,41.755701,7.498220,...,Arctic,47.893107,851.762924,30.536386,4.0,4.484241,2.167492e+09,7546,"{'Humans': 1.07421875, 'Dwarves': 14.74609375,...",4550000


In [48]:
def generate_random_list_data(columns, choices, size):
    data = {}
    for col, choice in zip(columns, choices):
        data[col] = [np.random.choice(choice, size=np.random.randint(1, len(choice)), replace=False).tolist() for _ in range(size)]
    return data


In [49]:


bounds = [
    (['City1', 'City2', 'City3'], 'object'),
    (['Region1', 'Region2', 'Region3'], 'object'),
    (['Nation1', 'Nation2', 'Nation3'], 'object'),
    (1000, 100000, 'int64'),
    (['Government1', 'Government2', 'Government3'], 'object'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0, 100, 'float64'),
    (['Old Age', 'Illness', 'Accident'], 'object'),
    (['School1', 'School2', 'School3'], 'object'),
    (0, 1e6, 'float64'),
    (0, 1e6, 'float64'),
    (0, 1e6, 'float64'),
    (0, 1e6, 'float64'),
    (0.0, 100.0, 'float64'),
    (['Industry1', 'Industry2', 'Industry3'], 'object'),
    (['Resource1', 'Resource2', 'Resource3'], 'object'),
    (0, 1000, 'int64'),
    (0, 1000, 'int64'),
    (0, 5000, 'int64'),
    (0, 10000, 'int64'),
    (0, 50000, 'int64'),
    (0, 1000, 'int64'),
    (0, 1000, 'int64'),
    (0, 1000, 'int64'),
    (0, 100, 'int64'),
    (0.0, 1.0, 'float64'),
    (0.0, 40.0, 'float64'),
    (0.0, 300.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (0.0, 100.0, 'float64'),
    (['Landmark1', 'Landmark2', 'Landmark3'], 'object'),
    (0, 100, 'int64')
]
size = 100

In [50]:
data = generate_random_uniform_data(columns, bounds, size)

In [53]:
 city_names = [
    "Abriymoch", "Aelor", "Ageadren", "Airspur", "Airspur (Chessenta)", "Akanax",
    "Akkaido", "Al-Anwahr", "Alashan", "Almraiven", "Ammathluir", "Ammathtar",
    "Anaxadren", "Araugul", "Ariel-than", "Asple", "Assam", "Athkatla",
    "Aventine", "Azumar", "Balanzia", "Baldur's Gate", "Beluir", "Beradandar",
    "Berydren", "Bhutan", "Bidnop So", "Blingdenstone", "Bral", "Branestria",
    "Callidae", "Calimport", "Castle Darkhold", "Cathyr", "Cedarsproke", "Chaoyang",
    "Chaul'mur'ssin", "Chavyondat", "Ched Nasad", "Cheju", "Choshein", "Chrysalis",
    "Chunming", "Chunming (T'u Lung)", "Cimbar", "Citadel of Ice and Steel",
    "Citadel of Ten Thousand Pearls", "City of Brass", "City of Caverns", "City of Glass",
    "City of the Werecrocodiles", "Clorlyth'na'tsheen", "Corrant", "Dalekeva",
    "Dalelost", "Darka", "Darmshall", "Darrawn", "Dashadjen", "Dauntshield",
    "Dawnfire", "Deepearth", "Delia", "Delselar", "Delzimmer", "Derlusk",
    "Dhinnilith", "Dilpur", "Djerad Kethendi", "Djerad Kusold", "Dloemen", "Dojyu",
    "Dolblunde", "Dralpur", "Drelagara", "Duhlnarim", "Dunspeirrin", "Dusklan",
    "Dyon G'ennivalz", "Elgrin Fau", "Ereshkigal", "Evereska", "Everlund", "Evernight",
    "Eylea", "Faenasuor", "Fatula Chupa", "Fij", "Fluvenilstra", "Flyndagol",
    "Fochu", "Foo Nakar", "Forharn", "Fukiow", "Gharreil", "Gheldaneth",
    "Gifu", "Gizekhtet (city)", "Glister", "Gloomwrought", "Gnarhelm (city)", "Goliad",
    "Gontal City", "Guallidurth", "Gundbarg", "Gurdeth", "Halagard", "Hammerstaad",
    "Harglast", "Haunvaerune", "Hazuth", "Helbrester", "Heldapan", "Helgabal",
    "Helm's Hold", "Herath", "Hethar", "Hilbrand", "Hisari", "Hlammach",
    "Huorm", "Iiso", "Ilimar", "Iliphanar", "Illul", "Iltkazar",
    "Imdolphyn", "Immurk's Hold", "In'ani", "Inupras", "Iriaebor", "Ironfang Deep",
    "Ironspur", "Ithmong (Lapaliiya)", "Ixtzul", "Jangling Hiter", "Jasuga", "Jhachalkhyn",
    "Jhalhoran", "Jhothûn (city)", "Joya", "Jugicha", "Kanaglym", "Karak",
    "Karffbadh", "Kaspar", "Keltar", "Khaerbaal", "Khampo", "Koje",
    "Konang Pen", "Konigheim", "Kormul", "Kourmira", "Kressilacc", "Kuda",
    "Kuo Te' Lung", "Kushk", "Lasdur", "Lheshayl", "Lipo",
    "Lith My'athar", "Llorbauth", "Llurth Dreier", "Lothen of the Silver Spires", "Luenath", "Lundeth (Thindol)",
    "Lushpool", "Luskan", "Lylorn", "Maerlar", "Malduir", "Malsheem",
    "Malydren", "Manass", "Marabaya", "Marangor", "Marrauk", "Maxal",
    "Melvaunt", "Messemprar", "Mhairhetel", "Mictlatepec", "Mithral Hall",
    "Minauros (city)", "Mirabar", "Mordulkin", "Morvar", "Motra", "Mreyelundar",
    "Mulmaster", "Multum", "Mussum", "Myrmyr", "Myth Adofhaer", "Myth Drannor",
    "Myth Nantar", "Nakamaru", "Narubel", "Nathong", "Neldorild", "Nesmé",
    "Nethjet", "Nethra", "Neverwinter", "New Sarshel", "Nivian", "Nonthal",
    "Northkeep", "Nycopolis", "Obbolor", "Oberon's Hold", "Occidian", "Oeble",
    "Ojy-do", "Okahira", "Old Stone Keep", "Old Vaelen", "Omu", "Ooltul",
    "Opawangchicheng", "Orlumbor", "Ormath", "Ormpetarr", "Ornthalas", "Oryndoll",
    "Osel", "Ossantil", "Oth", "Outentown", "Ozryn", "Paolan",
    "Paraelemental City of Conflagration", "Paraelemental City of Deluge", "Paraelemental City of Ooze",
    "Paraelemental City of Sleet", "Phandalin", "Phannaskul",
    "Phaervorul", "Phrenopolis", "Piaoliang", "Pinebluff", "Pirth City", "Piyapon",
    "Plague-Mort", "Plotsklan", "Port Castigliar", "Port Llast", "Port Nyanzaru", "Port of Ravens Bluff",
    "Poytakht", "Priapurl", "Procampur", "Purskul", "Pyarados", "Q'barra",
    "Q'Xorlarrin", "Qazat", "Quaervarr", "Quarmall", "Qu'ellar d'Uzunt", "Qu'ellarz'orl",
    "Radocan", "Raurokh", "Raven's Bluff", "Razalpartha", "Rekkar-Sarrat", "Relkath's Foot",
    "Reth", "Revonar", "Riatavin", "Rilauven", "Ruathym (city)", "Ruldoer",
    "Rylatar", "Saelmur", "Sakkors", "Saradush", "Sardolphyn", "Sarth",
    "Sattar", "Scardale", "Schamedar", "Schorin", "Secomber", "Sekras",
    "Selgaunt", "Semkhrun", "Sepech", "Serôs (city)", "Setma'antar", "Sethlapurkha",
    "Shaboath", "Shade (city)", "Shadusk", "Shangban", "Shaundalar", "Sheirtalar",
    "Shi-Kuo", "Shindikuur", "Shoonach", "Shu Pao", "Shussel", "Sildeyuir",
    "Siluvanede", "Simbia", "Sintharla", "Sjorndorn", "Skullport", "Skuld",
    "Soorenar", "Sschindylryn", "Sshamath", "Stornanter", "Sudar", "Sundabar",
    "Sunduluth", "Sungping", "Surkh", "Suzail", "Svartalfheim", "Taejin",
    "Tagorlad", "Takalai", "Takar", "Talab", "Talath Passar", "Talath Seren",
    "Talbot", "Talhund", "Talinie (city)", "Tammar", "Tantras", "Tasseldale",
    "Tchazzar", "Telantiwar", "Telflamm", "Telnarquel", "Telos City", "Telsorn",
    "Temar", "Termalaine", "Tethgard", "Thallar", "Thamor", "Thangorodrim",
    "Tharqualnaar", "Thayd", "Thelnam", "Thentia", "Thesk", "Thulanth",
    "Thultanthar", "Thundarlun", "Thundertree", "Thyat", "Tibet", "Tilverton",
    "Tith Tilendrothael", "Torsil", "Triboar", "Triel", "Tsurlagol", "Tu Pe",
    "Tulmon", "Tun Mi Lung", "Tuncer", "Tunfeng", "Tunzel", "Turmish (city)",
    "Twilight Hall", "Two Stars", "Tyar-Besil", "Ulgarth (city)", "Ulgarstat", "Umbergoth",
    "Umbraer", "Urkhar", "Urossos", "Uthmere", "Uzurr", "Vaasa (city)",
    "Valjevo Castle", "Vangal", "Velegrar", "Vilhon Reach", "Volothamp", "Voonlar",
    "Vor Rukoth", "V'ral", "Waterdeep", "Weeping War (city)", "Westgate", "Wheloon",
    "Xarcorr", "Xinlenal", "Xonthal's Tower", "Xynnar", "Yal Tengri", "Yaulazna",
    "Yhaunn", "Yhro 'Lamith", "Ylraphon", "Yuirwood", "Yunn Shicheng", "Za Si Tsan",
    "Zazesspur", "Zhentil Keep", "Zhenul", "Zindalankh", "Zokir"
    ]

In [54]:
if __name__ == "__main__":
    main()

TypeError: ignored

In [31]:
df.head()

,City,Region,Nation,Population of City,Government,Percentage of population Humans,Percentage of population Halflings,Percentage of population Orcs,Percentage of population Elves,Percentage of population Dwarves,...,Environmental Climate,Average Temperature,Average Rainfall,Political Climate,Magical Climate,Likelihood of Getting Cursed,Danger Level,Openness to Strangers,Notable Landmarks,Fate ranking
0,Abriymoch,Region3,Nation2,15922,Oligarchy,92.890664,7.904120,71.520153,80.264236,10.024334,...,21.149629,28.056975,51.715684,37.255145,16.583509,36.113485,90.531226,60.926050,Landmark1,45
1,Aelor,Region1,Nation3,12583,Oligarchy,76.311884,1.607618,11.346570,68.282074,80.864470,...,85.989144,42.689376,84.906626,15.165676,92.435707,33.064317,86.386139,5.308632,Landmark3,416
2,Ageadren,Region1,Nation2,84272,Oligarchy,12.111212,54.610246,89.830203,74.545746,30.843258,...,41.153760,21.158781,9.216342,65.065520,44.178035,94.057346,57.030169,46.249247,Landmark3,227
3,Airspur,Region2,Nation3,50224,Oligarchy,85.766203,73.721184,42.047838,30.094798,48.466543,...,47.134302,16.988302,23.385766,41.125886,54.351449,46.155325,35.290176,39.565244,Landmark2,86
4,Airspur (Chessenta),Region2,Nation3,17081,Oligarchy,9.291475,49.656559,84.705312,85.098654,94.914073,...,65.841312,78.579678,66.109504,50.585987,21.530267,12.120318,74.095791,33.795816,Landmark1,519


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   City                                411 non-null    object 
 1   Region                              411 non-null    object 
 2   Nation                              411 non-null    object 
 3   Population of City                  411 non-null    int64  
 4   Government                          411 non-null    object 
 5   Percentage of population Humans     411 non-null    float64
 6   Percentage of population Halflings  411 non-null    float64
 7   Percentage of population Orcs       411 non-null    float64
 8   Percentage of population Elves      411 non-null    float64
 9   Percentage of population Dwarves    411 non-null    float64
 10  Percentage of population Gnomes     411 non-null    float64
 11  Average Age of Population           411 non-n

In [14]:
# Save the updated DataFrame back to the Excel file
df.to_excel('/content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx', index=False)

In [15]:
# Step 1: Setting Total Population of Faerun
total_population = np.random.randint(66e6, 70e6)
cities = 411  # Number of cities
city_population = np.random.randint(100000, 300000, size=cities)
city_population = (city_population / city_population.sum()) * total_population

# Step 2: Setting Species Population Percentage for each city
df = pd.DataFrame()
df['City Population'] = city_population.astype(int)
df['Humans'] = np.random.uniform(0.78, 0.92, cities) * df['City Population']
df['Halflings'] = np.random.uniform(0.05, 0.06, cities) * df['City Population']
df['Dwarves'] = np.random.uniform(0.03, 0.04, cities) * df['City Population']
df['Elves'] = np.random.uniform(0.02, 0.03, cities) * df['City Population']
df['Orcs'] = np.random.uniform(0.02, 0.03, cities) * df['City Population']
df['Gnomes'] = np.random.uniform(0.01, 0.02, cities) * df['City Population']

# Convert them to integers
species_columns = ['Humans', 'Halflings', 'Dwarves', 'Elves', 'Orcs', 'Gnomes']
df[species_columns] = df[species_columns].astype(int)

# Print first few rows to check
print(df.head())

   City Population  Humans  Halflings  Dwarves  Elves  Orcs  Gnomes
0           211446  189768      11453     7622   5469  4244    3223
1           132049  117203       7846     4970   3219  3942    2361
2           102787   89566       5454     4031   2171  2598    1938
3           156526  126051       8041     5519   4484  3406    2897
4           207238  184331      12275     7835   5071  6053    3857


In [16]:
# Step 3: Setting Tax Rate and Number of Nobility
df['# of Nobility'] = (df['City Population'] * np.random.uniform(0.01, 0.03)).astype(int)
df['Tax Rate'] = np.random.uniform(0.1, 0.3, cities) + (df['# of Nobility'] / df['City Population'])

# Step 4: Calculating Danger Level
df['Dragon Sightings'] = np.random.randint(0, 100, cities)
df['Political Climate'] = np.random.uniform(0, 1, cities)
df['Magical Climate'] = np.random.uniform(0, 1, cities)
df['Openness to Strangers'] = np.random.uniform(0, 1, cities)
df['Average Debt by Household'] = np.random.uniform(0, 1e4, cities)
df['Rumored Treasure Value'] = np.random.uniform(0, 1e6, cities)

bottom_quartile = df['Openness to Strangers'].quantile(0.25)
top_quartile = df['Openness to Strangers'].quantile(0.75)
openness_score = np.where(df['Openness to Strangers'] < bottom_quartile, 1,
                np.where(df['Openness to Strangers'] > top_quartile, 1, 0))

df['Likelihood of Getting Cursed'] = df['Magical Climate'] + openness_score
df['Danger Level'] = (df['Dragon Sightings'] + df['Political Climate'] +
                             df['Magical Climate'] + df['Likelihood of Getting Cursed'] +
                             df['Tax Rate'] + df['Average Debt by Household'] +
                             df['Rumored Treasure Value'] + openness_score) / 8

# Step 5: Setting Economic Factors
total_economy = np.random.uniform(80e6, 100e6)
df['Active Economy (Gold)'] = (np.random.uniform(0.4, 0.6, cities) * (df['City Population'] / total_population) * total_economy).astype(int)
df['Hidden Economy (Gold)'] = ((df['City Population'] / total_population) * total_economy - df['Active Economy (Gold)']).astype(int)

# Step 6: Setting Population Count for Nobility and Peasants
df['# of Peasants'] = (df['City Population'] * np.random.uniform(0.8, 0.9)).astype(int)

# Step 7: Configuring Fate Ranking
df['Wizarding School'] = np.random.choice(['yes', 'no'], cities)
df['Fate ranking'] = (df['Rumored Treasure Value'] / 1e6) + np.where(df['Wizarding School'] == 'yes', 1, 0)

# Print first few rows to check
print(df.head())

   City Population  Humans  Halflings  Dwarves  Elves  Orcs  Gnomes  \
0           211446  189768      11453     7622   5469  4244    3223   
1           132049  117203       7846     4970   3219  3942    2361   
2           102787   89566       5454     4031   2171  2598    1938   
3           156526  126051       8041     5519   4484  3406    2897   
4           207238  184331      12275     7835   5071  6053    3857   

   # of Nobility  Tax Rate  Dragon Sightings  ...  Openness to Strangers  \
0           2633  0.292279                77  ...               0.453717   
1           1644  0.155011                19  ...               0.918264   
2           1280  0.157687                25  ...               0.514864   
3           1949  0.291992                38  ...               0.921383   
4           2581  0.121567                97  ...               0.653692   

   Average Debt by Household  Rumored Treasure Value  \
0                4605.786974           135209.679497   
1   

In [18]:
# Step 9: Setting Political Climate
df['# Homeowners'] = (df['City Population'] * np.random.uniform(0.5, 0.7)).astype(int)
df['Average Rent'] = np.random.uniform(500, 2000, cities)
df['Average Age of Population'] = np.random.uniform(25, 70, cities)
df['Known Treasure Value'] = np.random.uniform(0, 1e6, cities)

df['Political Climate'] = (df['Tax Rate'] + df['# Homeowners']/df['City Population'] +
                                  df['Average Rent']/2000 + df['Average Debt by Household']/1e4 +
                                  df['Average Age of Population']/70 + df['Hidden Economy (Gold)']/1e6 +
                                  df['Known Treasure Value']/1e6 + df['Rumored Treasure Value']/1e6) / 8

# Step 10: Adjusting Fate Ranking
df['Fate ranking'] = (df['Rumored Treasure Value'] / 1e6) + np.where(df['Wizarding School'] == 'yes', 1, 0) + df['Political Climate']

# Step 11: Making Dataset Generation Replicable
def generate_dataset(seed=42):
    np.random.seed(seed)
    # ... (insert all the previous code that generates the data here)
    return df

# Generate a new dataset with a different seed each time to get a slightly different dataset
new_dataset = generate_dataset(seed=45)
print(new_dataset.head())

# You can use the generate_dataset function to create new datasets with minor variations each time



   City Population  Humans  Halflings  Dwarves  Elves  Orcs  Gnomes  \
0           211446  189768      11453     7622   5469  4244    3223   
1           132049  117203       7846     4970   3219  3942    2361   
2           102787   89566       5454     4031   2171  2598    1938   
3           156526  126051       8041     5519   4484  3406    2897   
4           207238  184331      12275     7835   5071  6053    3857   

   # of Nobility  Tax Rate  Dragon Sightings  ...  Danger Level  \
0           2633  0.292279                77  ...  17486.726748   
1           1644  0.155011                19  ...  84428.834098   
2           1280  0.157687                25  ...  25978.069289   
3           1949  0.291992                38  ...  74876.358921   
4           2581  0.121567                97  ...  20239.987652   

   Active Economy (Gold)  Hidden Economy (Gold)  # of Peasants  \
0                 129080                 160374         180474   
1                  86123              

In [19]:
df.head()

,City,Region,Nation,Population of City,Government,Percentage of population Humans,Percentage of population Halflings,Percentage of population Orcs,Percentage of population Elves,Percentage of population Dwarves,...,Environmental Climate,Average Temperature,Average Rainfall,Political Climate,Magical Climate,Likelihood of Getting Cursed,Danger Level,Openness to Strangers,Notable Landmarks,Fate ranking
0,Abriymoch,Region2,Nation2,15922,Oligarchy,92.890664,7.904120,71.520153,80.264236,10.024334,...,21.149629,28.056975,51.715684,37.255145,16.583509,36.113485,90.531226,60.926050,Landmark1,45
1,Aelor,Region2,Nation3,12583,Oligarchy,76.311884,1.607618,11.346570,68.282074,80.864470,...,85.989144,42.689376,84.906626,15.165676,92.435707,33.064317,86.386139,5.308632,Landmark3,416
2,Ageadren,Region3,Nation2,84272,Oligarchy,12.111212,54.610246,89.830203,74.545746,30.843258,...,41.153760,21.158781,9.216342,65.065520,44.178035,94.057346,57.030169,46.249247,Landmark3,227
3,Airspur,Region2,Nation3,50224,Oligarchy,85.766203,73.721184,42.047838,30.094798,48.466543,...,47.134302,16.988302,23.385766,41.125886,54.351449,46.155325,35.290176,39.565244,Landmark2,86
4,Airspur (Chessenta),Region1,Nation3,17081,Oligarchy,9.291475,49.656559,84.705312,85.098654,94.914073,...,65.841312,78.579678,66.109504,50.585987,21.530267,12.120318,74.095791,33.795816,Landmark1,519


In [20]:
# Save the updated DataFrame back to the Excel file
df.to_excel('/content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx', index=False)

In [21]:
# Step 1: Update Percentage Columns
percentage_columns = [
    'Percentage of population Humans',
    'Percentage of population Halflings',
    'Percentage of population Dwarves',
    'Percentage of population Elves',
    'Percentage of population Orcs',
    'Percentage of population Gnomes'
]

for col in percentage_columns:
    df[col] = df[col].round(2)

KeyError: ignored

In [22]:
# Step 2: Adjust Danger Level

# Convert "Danger Level" to an integer between 1 and 10
df['Danger Level'] = (df['Dragon Sightings'] + df['Political Climate'] + df['Magical Climate'] + df['Likelihood of Getting Cursed'] + df['Tax Rate'] + df['Average Debt by Household'] + df['Rumored Treasure Value'] + df['Openness to Strangers'].apply(lambda x: 10 if x <= 2.5 or x >= 7.5 else 0)).astype(int)

# We normalize the "Danger Level" to be between 1 and 10
df['Danger Level'] = ((df['Danger Level'] - df['Danger Level'].min()) / (df['Danger Level'].max() - df['Danger Level'].min()) * 9 + 1).astype(int)

# Increase "Danger Level" if Orcs are present
df.loc[df['Percentage of population Orcs'] > 0, 'Danger Level'] += 1

# Decrease "Danger Level" if Elves are present
df.loc[df['Percentage of population Elves'] > 0, 'Danger Level'] -= 1

# Ensure "Danger Level" remains within the range 1-10
df['Danger Level'] = df['Danger Level'].clip(1, 10)


KeyError: ignored

In [23]:
# Step 3: Modify Dragon Sightings

# First, we'll create a mapping of the total gold per region
region_gold_mapping = df.groupby('Region')['Active Economy (Gold)', 'Hidden Economy (Gold)'].sum().sum(axis=1)

# Next, we'll adjust the Dragon Sightings based on the total gold in the region and the Fate ranking of the city
df['Dragon Sightings'] = df.apply(lambda row: (row['Fate ranking'] * (region_gold_mapping[row['Region']] / 1e6)), axis=1)

# We'll then normalize this to be a number between 0 and 50
df['Dragon Sightings'] = ((df['Dragon Sightings'] - df['Dragon Sightings'].min()) / (df['Dragon Sightings'].max() - df['Dragon Sightings'].min()) * 50).astype(int)

# Making sure "Dragon Sightings" does not exceed 50
df['Dragon Sightings'] = df['Dragon Sightings'].clip(0, 50)


KeyError: ignored

In [24]:
# Step 4: Determine Average Age of Population

# First, we'll define a function to calculate the average age based on the population percentages and the species' age ranges
def calculate_average_age(row):
    species_age_ranges = {
        "Humans": [1, 100],
        "Dwarves": [1, 350],
        "Elves": [1, 750],
        "Gnomes": [300, 500],
        "Halflings": [1, 150],
        "Orcs": [1, 50]
    }

    total_population = row['Population of City']
    weighted_ages = sum([(row[f'Percentage of population {species}'] / 100) * (sum(species_age_ranges[species]) / 2) * total_population for species in species_age_ranges])

    return weighted_ages / total_population

# Now, we'll apply this function to determine the average age of population
df['Average Age of Population'] = df.apply(calculate_average_age, axis=1)

# Considering the effect of "Danger Level" on the average age
df['Average Age of Population'] = df['Average Age of Population'] - (df['Danger Level'] * 0.5)

# Ensuring the average age doesn't fall below the minimum possible age
df['Average Age of Population'] = df['Average Age of Population'].clip(1)


KeyError: ignored

In [25]:
import numpy as np

# Adjusting population distribution for specific species across cities
def adjust_population_distribution(df, species, percentage, city_count):
    total_population = df['Population of City'].sum()
    target_population = int(total_population * (percentage / 100))
    selected_cities = df.sample(city_count)
    each_city_population = target_population // city_count
    for i, city in selected_cities.iterrows():
        df.at[i, f'Percentage of population {species}'] = (each_city_population / city['Population of City']) * 100

# Adjusting the population distribution
adjust_population_distribution(df, "Gnomes", 99, 2)
adjust_population_distribution(df, "Dwarves", 80, 4)
adjust_population_distribution(df, "Elves", 95, 5)
adjust_population_distribution(df, "Orcs", 99, 1)

# Adjusting Dragon Sightings
df['Dragon Sightings'] = (df['Region'].map(df.groupby('Region')['Fate ranking'].sum() + df.groupby('Region')['Active Economy (Gold)'].sum()) / 1e6).clip(0, 50)

# Adjusting Danger Level
df['Danger Level'] = df.apply(lambda row: np.random.randint(1, 11) + (3 if row['Percentage of population Orcs'] > 0 else 0) - (2 if row['Percentage of population Elves'] > 0 else 0), axis=1)

# Adjusting Openness to Strangers
df['Openness to Strangers'] = df.apply(lambda row: np.random.randint(0, 11) - (3 if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0 else 0), axis=1)

# Adjusting Magical Climate
df['Magical Climate'] = df.apply(lambda row: row['Magical Climate'] + (row['Percentage of population Elves'] / 100 * 2) + (row['Dragon Sightings'] / 10) + row['Number of Active Wizards'], axis=1)

# Adjusting Hidden Economy based on the number of nobility
df['Hidden Economy (Gold)'] = df.apply(lambda row: row['Hidden Economy (Gold)'] * (1 + (row['# of Nobility'] / row['Population of City']) * 100), axis=1)

# Ensuring Percentage values are rounded to 2 decimal places
percentage_columns = [col for col in df.columns if 'Percentage' in col]
df[percentage_columns] = df[percentage_columns].round(2)

# Making sure Openness to Strangers is between 0 and 10
df['Openness to Strangers'] = df['Openness to Strangers'].clip(0, 10)


KeyError: ignored

In [26]:
def calculate_average_age(row):
    human_percentage = row['Percentage of population Humans']
    dwarf_percentage = row['Percentage of population Dwarves']
    elf_percentage = row['Percentage of population Elves']
    gnome_percentage = row['Percentage of population Gnomes']
    halfling_percentage = row['Percentage of population Halflings']
    orc_percentage = row['Percentage of population Orcs']

    # Getting the random average age for each species
    human_avg_age = np.random.uniform(1, 100, 1)[0]
    dwarf_avg_age = np.random.uniform(1, 350, 1)[0]
    elf_avg_age = np.random.uniform(1, 750, 1)[0]
    gnome_avg_age = np.random.uniform(300, 500, 1)[0]
    halfling_avg_age = np.random.uniform(1, 150, 1)[0]
    orc_avg_age = np.random.uniform(1, 50, 1)[0]

    # Calculating the average age of the population
    average_age = (
        (human_avg_age * human_percentage) +
        (dwarf_avg_age * dwarf_percentage) +
        (elf_avg_age * elf_percentage) +
        (gnome_avg_age * gnome_percentage) +
        (halfling_avg_age * halfling_percentage) +
        (orc_avg_age * orc_percentage)
    ) / 100

    return average_age

# Setting the average age of population
df['Average Age of Population'] = df.apply(calculate_average_age, axis=1)

# Setting the number of active wizards randomly between 0 and 20
df['Number of Active Wizards'] = np.random.randint(0, 21, df.shape[0])


KeyError: ignored

In [27]:
def calculate_dragon_sightings(row):
    # The dragon sightings are influenced by the total gold in a region and the fate ranking of the city
    region_gold = df[df['Region'] == row['Region']]['Active Economy (Gold)'].sum() + df[df['Region'] == row['Region']]['Hidden Economy (Gold)'].sum()
    max_sightings = min(50, row['Fate ranking'] + region_gold // 1e6)
    return np.random.randint(0, max_sightings+1)

def adjust_danger_level(row):
    # Adjusting danger level based on the presence of Orcs and Elves
    danger_level = row['Danger Level']
    if row['Percentage of population Orcs'] > 0:
        danger_level += 1
    if row['Percentage of population Elves'] > 0:
        danger_level -= 1
    return min(max(1, danger_level), 10)

def calculate_magical_climate(row):
    # Adjusting magical climate based on dragon sightings and the number of active wizards
    return row['Dragon Sightings'] * 0.1 + row['Number of Active Wizards'] * 0.5

def calculate_openness_to_strangers(row):
    # Adjusting openness to strangers based on the presence of Elves or Orcs
    openness = row['Openness to Strangers']
    if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0:
        openness -= 2
    return min(max(0, openness), 10)

# Applying the functions
df['Dragon Sightings'] = df.apply(calculate_dragon_sightings, axis=1)
df['Danger Level'] = df.apply(adjust_danger_level, axis=1)
df['Magical Climate'] = df.apply(calculate_magical_climate, axis=1)
df['Openness to Strangers'] = df.apply(calculate_openness_to_strangers, axis=1)


KeyError: ignored

In [28]:
# Step 8.1: Adjusting Population Distribution
# Define the cities where majority of the specific species will reside
gnome_cities = np.random.choice(df['City'], size=2, replace=False)
dwarf_cities = np.random.choice(df['City'], size=4, replace=False)
elf_cities = np.random.choice(df['City'], size=5, replace=False)
orc_city = np.random.choice(df['City'], size=1, replace=False)

# Define functions to adjust the population percentages for Gnomes, Dwarves, Elves, and Orcs
def adjust_gnome_population(row):
    if row['City'] in gnome_cities:
        return np.random.uniform(40, 50)
    return np.random.uniform(0, 0.1)

def adjust_dwarf_population(row):
    if row['City'] in dwarf_cities:
        return np.random.uniform(15, 25)
    return np.random.uniform(0, 1)

def adjust_elf_population(row):
    if row['City'] in elf_cities:
        return np.random.uniform(15, 20)
    return np.random.uniform(0, 1)

def adjust_orc_population(row):
    if row['City'] == orc_city:
        return np.random.uniform(90, 99)
    return np.random.uniform(0, 1)

# Apply the functions to adjust the population distribution
df['Percentage of population Gnomes'] = df.apply(adjust_gnome_population, axis=1)
df['Percentage of population Dwarves'] = df.apply(adjust_dwarf_population, axis=1)
df['Percentage of population Elves'] = df.apply(adjust_elf_population, axis=1)
df['Percentage of population Orcs'] = df.apply(adjust_orc_population, axis=1)

# Step 8.2: Calculating Average Age of Population
# Define a function to calculate the average age based on the species population percentages and danger level
def calculate_average_age(row):
    human_age = np.random.uniform(1, 100)
    dwarf_age = np.random.uniform(1, 350)
    elf_age = np.random.uniform(1, 750)
    gnome_age = np.random.uniform(300, 500)
    halfling_age = np.random.uniform(1, 150)
    orc_age = np.random.uniform(1, 50)
    total_population = row['Total Population']

    avg_age = ((human_age * row['Percentage of population Humans'] +
                dwarf_age * row['Percentage of population Dwarves'] +
                elf_age * row['Percentage of population Elves'] +
                gnome_age * row['Percentage of population Gnomes'] +
                halfling_age * row['Percentage of population Halflings'] +
                orc_age * row['Percentage of population Orcs']) / 100)

    return avg_age - (row['Danger Level'] / 10)

# Apply the function to calculate the average age
df['Average Age of Population'] = df.apply(calculate_average_age, axis=1)


KeyError: ignored

In [29]:
# Step 9.1: Calculate Dragon Sightings based on total gold in a region and the Fate ranking of the city
df['Dragon Sightings'] = (df['Total Gold in Region'] / 1e6) + df['Fate ranking']
df['Dragon Sightings'] = df['Dragon Sightings'].apply(lambda x: min(x, 50))

# Step 9.2: Modify Danger Level
def calculate_danger_level(row):
    danger_level = row['Danger Level']
    if row['Percentage of population Orcs'] > 0:
        danger_level += 1
    if row['Percentage of population Elves'] > 0:
        danger_level -= 1
    return min(max(int(danger_level), 1), 10)
df['Danger Level'] = df.apply(calculate_danger_level, axis=1)

# Step 9.3: Modify Magical Climate
def calculate_magical_climate(row):
    magical_climate = row['Magical Climate']
    if row['Percentage of population Elves'] > 0:
        magical_climate += 1
    magical_climate += row['Dragon Sightings'] / 10
    magical_climate += row['Number of Active Wizards'] / 2
    return min(max(int(magical_climate), 1), 10)
df['Magical Climate'] = df.apply(calculate_magical_climate, axis=1)

# Step 9.4: Modify Openness to Strangers
def calculate_openness_to_strangers(row):
    openness = row['Openness to Strangers']
    if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0:
        openness -= 2
    return min(max(int(openness), 0), 10)
df['Openness to Strangers'] = df.apply(calculate_openness_to_strangers, axis=1)

# Step 9.5: Modify Hidden Economy
df['Hidden Economy'] = df['# of Nobility'] * 1e4


KeyError: ignored